# Cloning the Fine Tuned Model From my GitHub


In [1]:
!git clone https://github.com/karthiKN-sk/grootan_ai_task.git

Cloning into 'grootan_ai_task'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 71 (delta 27), reused 27 (delta 2), pack-reused 8 (from 2)
Receiving objects: 100% (71/71), 154.16 MiB | 18.06 MiB/s, done.
Resolving deltas: 100% (27/27), done.
Updating files: 100% (6/6), done.


# Download the Library/Packages

In [2]:
!pip3 install opencv-python ultralytics supervision numpy pandas matplotlib rich.progress transformers gradio huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 131.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211

# Import Libraries

In [3]:
from ultralytics import YOLO
import supervision as sv
import cv2
import numpy as np
import matplotlib.pyplot as plt
from rich.progress import Progress
from typing import Dict, Iterable, List, Optional, Set, Any
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import os
from huggingface_hub import login

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


# Setting HF Token in Environment variable

In [4]:

token = None
with open("/content/grootan_ai_task/variables.py", "r") as f:
    for line in f:
        if line.startswith("HF_TOKEN="):
            token = line.strip().split("=", 1)[1]
            break
os.environ["HF_TOKEN"]= token

login(token=os.environ.get("HF_TOKEN"))

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


# Configuration for Zone Definitions, Color Palette, and Vehicle Turn Classification.

This code snippet sets up essential constants and configurations for a vehicle tracking and turn analysis system, including color settings, polygonal zone definitions, naming schemes, turn mapping logic, and a utility function for turn-based color annotation.

1. Color Palette Definition
    * Defines a reusable color palette used for drawing zones, bounding boxes, and labels.

2. Zone Definitions (as Polygons)

    * ZONE_IN_POLYGONS: List of polygonal coordinates marking entry zones for vehicles.

    * ZONE_OUT_POLYGONS: List of polygonal coordinates marking exit zones for vehicles.

    * Each polygon is an array of 2D points (x, y).

3. Zone Naming
    * Provides readable labels for each entry and exit zone, mapped in order to the defined polygons.
4. Vehicle Turn Color Function(**get_color_for_turn**)

    * Returns a specific color for each type of turn:

        * Right Turn → Red

        * Left Turn → Green

        * U-turn → Black

        * Straight → Blue
5. Turn Mapping Logic

      * Maps combinations of entry (In1,2,3,4) and exit (Out1,2,3,4) zones to specific turn types (right_turn, left_turn, u_turn, straight).

      * Used to classify vehicle movement patterns across zones.
      
This setup provides a foundational configuration layer for visual annotation, spatial zone management, and logical turn analysis in a computer vision-based traffic monitoring system.

In [5]:
COLORS = sv.ColorPalette.from_hex(["#E6194B", "#3CB44B", "#FFE119", "#3C76D1"])

ZONE_IN_POLYGONS = [
    np.array([[1220, 360], [1500,600], [1600, 420], [1350, 200]]),
    np.array([[680, 260], [820, 380], [1020, 220], [880, 80]]),
    np.array([[480, 700], [680, 900], [800, 800], [660, 600]]),
    np.array([[1180, 1060], [1380, 880], [1200, 740], [1050, 1010]]),
]

ZONE_OUT_POLYGONS = [
    np.array([[950, 120], [1200, 350], [1340, 180], [1160, 20]]),
    np.array([[650, 260], [420, 480], [560, 620], [820, 400]]),
    np.array([[680, 920], [850, 1050], [1050, 1000], [820, 800]]),
    np.array([[1380, 860],[1220, 720],[1500, 620],[1640, 720],]),
]

ZONE_IN_NAMES = ["In1", "In2", "In3", "In4"]
ZONE_OUT_NAMES = ["Out1", "Out2", "Out3", "Out4"]

def get_color_for_turn(tracker_id, vehicle_turns):
    turn = vehicle_turns.get(tracker_id)
    if turn == "right_turn":
        return sv.Color.RED
    elif turn == "left_turn":
        return sv.Color.GREEN
    elif turn == "u_turn":
        return sv.Color.BLACK
    else:
        return sv.Color.BLUE

TURN_MAPPING = {
    "In1": {"Out2": "right_turn", "Out4": "left_turn", "Out3": "straight", "Out1": "u_turn"},
    "In2": {"Out3": "right_turn", "Out1": "left_turn", "Out4": "straight", "Out2": "u_turn"},
    "In3": {"Out4": "right_turn", "Out2": "left_turn", "Out1": "straight", "Out3": "u_turn"},
    "In4": {"Out1": "right_turn", "Out3": "left_turn", "Out2": "straight", "Out4": "u_turn"},
}

# Global Detection State Initialization for Vehicle Turn Tracking.

This snippet initializes a global dictionary named detections_state that is used to persist and manage tracking data across video frames in a vehicle monitoring pipeline.

Dictionary Keys:

* "**tracker_id_to_zone_id**":
   Maps each unique vehicle (tracker_id) to the zone ID it first appeared in.
   Used to group and identify vehicles during processing.

* "**vehicle_paths**":
Tracks both the entry (in) and exit (out) zones for each vehicle.
Format: **{tracker_id: {"in": zone_name, "out": zone_name}}**.

* "**vehicle_turns**":
Stores the classified type of turn for each vehicle, such as "left_turn", "right_turn", "u_turn", or "straight".

This stateful object is referenced and updated throughout the processing pipeline to enable accurate vehicle path tracking and turn classification across video frames.

In [6]:
detections_state = {
    "tracker_id_to_zone_id": {},
    "vehicle_paths": {},
    "vehicle_turns": {},
}

# Vehicle Entry-Exit Tracking and Turn Classification Logic

This function, update_detections_state, manages and updates the internal state used to track vehicle movement through predefined zones, and classifies the type of turn each vehicle makes (left, right, U-turn, or straight).

Key Responsibilities:

1. Track Entry Zones:

    * Maps each vehicle (via its tracker ID) to the in zone where it first appeared.

    * Ensures the entry point is recorded only once per vehicle.

2. Track Exit Zones:

    * Detects and records the out zone where the vehicle exits.

3. Turn Detection:

    * Uses predefined TURN_MAPPING to determine the turn type based on zone pairs (in → out).

    * Updates the vehicle_turns dictionary only when both zones are known.

4. Class ID Assignment:

    * Associates each detection with a zone ID for visual annotation by mapping tracker_id to its zone_id.

5. Filtering Valid Detections:

    * Returns only detections that have been successfully associated with zones (i.e., not class ID -1).

This function plays a central role in transforming low-level detection data into high-level vehicle movement understanding, essential for turn analysis in traffic videos.

In [7]:
def update_detections_state(
    detections_all: sv.Detections,
    detections_in_zones: List[sv.Detections],
    detections_out_zones: List[sv.Detections],
    config: Dict[str, Any],
    state: Dict[str, Any] = detections_state
) -> sv.Detections:
    tracker_id_to_zone_id = state["tracker_id_to_zone_id"]
    vehicle_paths = state["vehicle_paths"]
    vehicle_turns = state.setdefault("vehicle_turns", {})

    # --- Assign entry zones and track vehicle IN zone names ---
    zone_in_names = list(config["zones_in"].keys())  # cache the keys list
    for zone_in_id, detections_in_zone in enumerate(detections_in_zones):
        zone_name = zone_in_names[zone_in_id]
        for tracker_id in detections_in_zone.tracker_id:
            tracker_id_to_zone_id.setdefault(tracker_id, zone_in_id)

            # Initialize vehicle path if not already present
            vehicle_paths.setdefault(tracker_id, {"in": None, "out": None})
            if vehicle_paths[tracker_id]["in"] is None:
                vehicle_paths[tracker_id]["in"] = zone_name

    # --- Count exits grouped by entry zone and track vehicle OUT zone names ---
    zone_out_names = list(config["zones_out"].keys())
    for zone_out_id, detections_out_zone in enumerate(detections_out_zones):
        zone_name = zone_out_names[zone_out_id]
        for tracker_id in detections_out_zone.tracker_id:
            if tracker_id in tracker_id_to_zone_id:
                vehicle_paths.setdefault(tracker_id, {"in": None, "out": None})
                if vehicle_paths[tracker_id]["out"] is None:
                    vehicle_paths[tracker_id]["out"] = zone_name

    # --- Detect turns ---
    for tracker_id, path in vehicle_paths.items():
        in_zone = path["in"]
        out_zone = path["out"]
        if in_zone and out_zone and tracker_id not in vehicle_turns:
            turn_type = TURN_MAPPING.get(in_zone, {}).get(out_zone)
            if turn_type:
                vehicle_turns[tracker_id] = turn_type

    # Assign class_id for drawing/annotation
    if len(detections_all) > 0:
        detections_all.class_id = np.vectorize(
            lambda x: tracker_id_to_zone_id.get(x, -1)
        )(detections_all.tracker_id)
    else:
        detections_all.class_id = np.array([], dtype=int)

    return detections_all[detections_all.class_id != -1]


# Zone Initialization and Centroid Calculation Utilities.

This code provides two utility functions used in the vehicle turn detection system:
1. **initiate_polygon_zones**:

    * Takes a list of zone names and corresponding polygon coordinates.
    * Initializes and returns a dictionary mapping each name to a PolygonZone object.
    * Each PolygonZone uses the specified polygon shape and a list of triggering_anchors (default: CENTER) to detect when vehicles enter the zone.

2. **compute_centroid**:

    * Computes the centroid (geometric center) of a given polygon using the mean of its vertex coordinates.
    * Returns the result as a sv.Point, which can be used for positioning labels or visual markers in the frame.

These functions are key to defining spatial zones for detecting vehicle movement and labeling them appropriately in the video annotation process.

In [8]:

def initiate_polygon_zones(
    names: List[str],
    polygons: List[np.ndarray],
    triggering_anchors: Iterable[sv.Position] = [sv.Position.CENTER],
) -> Dict[str, sv.PolygonZone]:
    return {
        name: sv.PolygonZone(polygon=polygon, triggering_anchors=triggering_anchors)
        for name, polygon in zip(names, polygons)
    }

def compute_centroid(polygon):
    """Compute the centroid (mean point) of a polygon."""
    centroid = np.mean(polygon, axis=0).astype(int)
    return sv.Point(*centroid)

# Setup Configuration for Vehicle Turn Detection Pipeline.

The **setup_video_processor** function initializes and returns a configuration dictionary containing all essential components and parameters needed to process a video for vehicle turn detection. Here's what it sets up:

1. Video Input/Output Paths:

      * **source_video_path**: Path to the input video.

      * **target_video_path**: Optional path to save the processed output.

2. Detection Parameters:

      * **confidence_threshold**: Minimum confidence level for YOLO model detections.

      * **iou_threshold**: IOU threshold used during object tracking.

3. Detection and Tracking Tools:

      * **model**: A fine-tuned YOLO model for vehicle detection.

      * **tracker**: ByteTrack tracker for maintaining vehicle identities.

4. Video Metadata:

      * **video_info**: Extracts frame rate, resolution, and frame count from the input video.

5. Zone Definitions:

      * **zones_in** and **zones_out**: Defined polygon areas to detect entry and exit for turn classification.

6. Annotation Tools:

      * **box_annotator**: Draws bounding boxes on detected vehicles.

      * **label_annotator**: Displays vehicle IDs.

      * **trace_annotator**: Adds trajectory traces to show vehicle movement paths.

7. Detection State Handler:

      * **detections_manager**: A function to manage turn state updates.

This setup function centralizes the configuration, making it easy to pass all necessary components to the video processing pipeline.

In [9]:
def setup_video_processor(
    source_video_path: str,
    target_video_path: Optional[str] = None,
    confidence_threshold: float = 0.4,
    iou_threshold: float = 0.7,
) -> Dict[str, Any]:
    return {
        "conf_threshold": confidence_threshold,
        "iou_threshold": iou_threshold,
        "source_video_path": source_video_path,
        "target_video_path": target_video_path,
        "model": YOLO("/content/grootan_ai_task/models/YoloFineTunedV1.pt"),
        "tracker": sv.ByteTrack(),
        "video_info": sv.VideoInfo.from_video_path(source_video_path),
        "zones_in": initiate_polygon_zones(ZONE_IN_NAMES,ZONE_IN_POLYGONS),
        "zones_out": initiate_polygon_zones(ZONE_OUT_NAMES,ZONE_OUT_POLYGONS),
        "box_annotator": sv.BoxAnnotator(color=COLORS),
        "label_annotator": sv.LabelAnnotator(color=COLORS, text_color=sv.Color.BLACK),
        "trace_annotator": sv.TraceAnnotator(
            color=COLORS, position=sv.Position.CENTER, trace_length=100, thickness=2
        ),
        "detections_manager": update_detections_state,
    }

# Analyze and Visualize Vehicle Turn Statistics

The **analyze_turns** function evaluates vehicle turn data to generate a summary of turn behavior and visual insights. Here's what it does:

1. Summary Computation:
    *  Counts total tracked vehicles.
    *  Computes how many made right turns, left turns, U-turns, or went straight.

2. Console Output:
    *   Prints a summary report of the turn counts to the terminal.

3. Data Packaging:
    *   Creates a JSON-style dictionary (**turn_message**) containing:
        *   A message,
        *   Overall turn statistics (**turn_counts**),
        *   Individual vehicle turn details by tracker ID (**turn_details**).


4. Visualization:

      *   Plots a bar chart using **matplotlib** to visually represent the count of each turn type.
      *   Saves the chart as **turn_analysis.png** for later use (e.g., appending to video).

5. Return:
      *   Outputs the structured **turn_message**, suitable for downstream use in reports or QA systems.

This function bridges raw detection data with user-friendly output, supporting both analysis and visualization of vehicle behavior.


In [10]:
def analyze_turns(vehicle_turns):
    """Analyze the turns and create summary statistics"""
    total_vehicles = len(vehicle_turns)
    if total_vehicles == 0:
        print("No vehicles were detected or tracked.")
        return
    # Count the different types of turns
    right_turns = sum(1 for turns in vehicle_turns.values() if turns == "right_turn" )
    left_turns = sum(1 for turns in vehicle_turns.values() if turns == "left_turn" )
    u_turns = sum(1 for turns in vehicle_turns.values() if turns == "u_turn" )
    no_turns = sum(1 for turns in vehicle_turns.values() if turns == "straight")

    print("\n--- Turn Analysis Results ---")
    print(f"Total unique vehicles tracked: {total_vehicles}")
    print(f"Vehicles making right turns: {right_turns} ")
    print(f"Vehicles making left turns: {left_turns}")
    print(f"Vehicles making U-turns: {u_turns}")
    print(f"Vehicles with no detected turns: {no_turns}")


    # Create a visualization
    turn_counts = {
        'Right Turn': right_turns,
        'Left Turn': left_turns,
        'U-Turn': u_turns,
        'No Turn': no_turns
    }

    turn_message = {
    "message": "Turn Analysis Results completed.",
    "total_vehicles": total_vehicles,
    "turn_counts": {
        "Vehicles making right turns" : right_turns,
        "Vehicles making left turns": left_turns,
        "Vehicles making U-turns": u_turns,
        "Vehicles with no detected turns (Straight)": no_turns
    },
     "turn_details": [
        {"tracker_id": tracker_id, "turn": turn}
        for tracker_id, turn in vehicle_turns.items()
     ]
    }

    plt.figure(figsize=(10, 6))
    colors = ['red', 'green', 'black', 'blue']
    plt.bar(turn_counts.keys(), turn_counts.values(), color=colors)
    plt.title('Vehicle Turn Analysis')
    plt.ylabel('Number of Vehicles')
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    # Add count labels on top of each bar
    for i, (key, value) in enumerate(turn_counts.items()):
        plt.text(i, value + 0.3, str(value), ha='center')

    plt.savefig('turn_analysis.png')
    plt.show()
    plt.close()
    return turn_message

# Process and Annotate Video Frames for Vehicle Turn Detection

The **process_video** function reads a video frame-by-frame, processes each frame to detect and annotate vehicles, and outputs the results either to a video file or a live display window. Here's what it does:

1. Frame Extraction: Uses a frame generator to read frames from the source video.

2. Progress Tracking: Displays a live progress bar using the rich library to monitor video processing.

3. Annotation Pipeline:
    *   For each frame, it calls **process_frame**() to detect vehicles, determine turn behavior, and apply annotations.

4. Output Handling:

    *   If output path is specified: Saves the annotated video to disk using VideoSink.
    *   Otherwise: Displays annotated frames live using OpenCV (**cv2.imshow**).

5. Sample Frame Export: Saves a single annotated frame as an image (annotated_output.png) for preview or debugging.

6. Returns the dictionary of vehicle turn states (vehicle_turns) tracked during the video processing.

This function is the core executor of the turn detection pipeline, enabling both real-time display and file output of the analyzed results.

In [11]:
def process_video(config: Dict[str, Any]) -> None:
    frame_generator = sv.get_video_frames_generator(config["source_video_path"])
    total = config["video_info"].total_frames

    with Progress() as progress:
        task = progress.add_task("[green]Processing video...", total=total)
        if config["target_video_path"]:
            with sv.VideoSink(config["target_video_path"], config["video_info"]) as sink:
                saved_sample = False
                for frame in frame_generator:
                    annotated = process_frame(frame, config)
                    sink.write_frame(annotated)
                    if not saved_sample:
                        cv2.imwrite("annotated_output.png", annotated)
                        saved_sample = True
                    progress.advance(task)
        else:
            for frame in frame_generator:
                annotated = process_frame(frame, config)
                cv2.imshow("Processed Video", annotated)
                if cv2.waitKey(1) & 0xFF == ord("q"):
                    break
                progress.advance(task)
            cv2.destroyAllWindows()

    return detections_state["vehicle_turns"]

# Annotate Video Frames with Zone Information and Vehicle Turn Statistics


The **annotate_frame** function overlays comprehensive visual annotations on each video frame to aid in understanding vehicle movement and behavior. Here's what it does:

1. Draws entry and exit zones (**zones_in and zones_out**) on the frame using polygons and labels with distinct colors.

2. Generates labels for each detected vehicle using their tracker IDs (e.g., **"Car #12"**).

3. Applies multiple annotation layers:
    *   Trajectory traces via **trace_annotator**
    *   Bounding boxes via **box_annotator**
    *   Vehicle ID labels via **label_annotator**

4. Computes turn statistics from the global detections_state:
    *   Total vehicles tracked
    *   Counts of right turns, left turns, U-turns, and straight movements

5. Displays summary metrics visually on the frame, including:
    *   A detection count badge
    *   Fixed-position statistics on turn types, color-coded for clarity (e.g., red for right turns, green for left turns, etc.)

This function is central to making the video output interpretable by overlaying both spatial (zones) and behavioral (turn types) information for each detected vehicle.

In [12]:
def annotate_frame(frame: np.ndarray, detections: sv.Detections, config: Dict[str, Any]) -> np.ndarray:
    frame_ = frame.copy()

    # Draw zones
    for i, ((zin_name, zin), (zout_name, zout)) in enumerate(zip(config["zones_in"].items(), config["zones_out"].items())):
        color = COLORS.colors[i % len(COLORS.colors)]
        zin_anchor = compute_centroid(zin.polygon)
        zout_anchor = compute_centroid(zout.polygon)
        frame_ = sv.draw_polygon(frame_, zin.polygon, color)
        frame_ = sv.draw_text(frame_, text=zin_name, text_anchor=zin_anchor, text_color=color)
        frame_ = sv.draw_polygon(frame_, zout.polygon, color)
        frame_ = sv.draw_text(frame_, text=zout_name, text_anchor=zout_anchor, text_color=color)

    labels = [f"Car #{id_}" for id_ in detections.tracker_id]

    frame_ = config["trace_annotator"].annotate(frame_, detections)
    frame_ = config["box_annotator"].annotate(frame_, detections)
    frame_ = config["label_annotator"].annotate(frame_, detections, labels)

    # Count the different types of turns
    vehicle_turns= detections_state["vehicle_turns"]
    total_vehicles = len(vehicle_turns)
    right_turns = sum(1 for turns in vehicle_turns.values() if turns == "right_turn" )
    left_turns = sum(1 for turns in vehicle_turns.values() if turns == "left_turn" )
    u_turns = sum(1 for turns in vehicle_turns.values() if turns == "u_turn" )
    no_turns = sum(1 for turns in vehicle_turns.values() if turns == "straight")

    # Add detection count info
    total_count = len(detections)
    frame_ = sv.draw_text(
        frame_,
        f"Detected: {total_count}",
        sv.Point(50, 50),
        background_color=sv.Color.from_hex("#FF7F50")
    )
    # Draw fixed turn statistics on the center-left of the frame
    start_x = 80
    start_y = 350
    line_spacing = 40
    text_color = sv.Color(r=255, g=255, b=255)

    # Line 1: Total vehicles tracked
    frame_ = sv.draw_text(
        frame_,
        text=f"Total vehicles tracked: {total_vehicles}",
        text_anchor=sv.Point(start_x + 30, start_y),
        background_color=sv.Color.from_hex("#DDDDDD"),
    )

    # Line 2: Right turns (Red)
    frame_ = sv.draw_text(
        frame_,
        text=f"Right turns: {right_turns}",
        text_anchor=sv.Point(start_x + 10, start_y + line_spacing),
        background_color=sv.Color(r=255, g=0, b=0),
        text_color=text_color
    )

    # Line 3: Left turns (Green)
    frame_ = sv.draw_text(
        frame_,
        text=f"Left turns: {left_turns}",
        text_anchor=sv.Point(start_x + 10, start_y + 2 * line_spacing),
        background_color=sv.Color(r=0, g=255, b=0),

    )

    # Line 4: U-turns (Black)
    frame_ = sv.draw_text(
        frame_,
        text=f"U-turns: {u_turns}",
        text_anchor=sv.Point(start_x + 10, start_y + 3 * line_spacing),
        background_color=sv.Color(r=0, g=0, b=0),
        text_color=text_color
    )

    # Line 5: No turns (Blue)
    frame_ = sv.draw_text(
        frame_,
        text=f"No turns: {no_turns}",
        text_anchor=sv.Point(start_x + 10, start_y + 4 * line_spacing),
        background_color=sv.Color(r=0, g=0, b=255),
        text_color=text_color
    )


    return frame_

# Process and Annotate Video Frame for Vehicle Turn Detection

The function **process_frame** handles a single video frame in the vehicle turn detection pipeline. Here's a breakdown of its functionality:

1. Runs object detection on the input frame using a **YOLO model (from config["model"])**, with specified confidence and IoU thresholds.

2. Converts detection results into a standardized format (**sv.Detections**) and forces all detected class IDs to zero (indicating a single-class tracking scenario, like vehicles).

3. Updates object tracks using a tracking algorithm (**config["tracker"]**).

4. Checks zone entry/exit: For each pair of entry (**zone_in**) and exit (**zone_out**) zones, it filters detections currently inside these zones.

5. Filters detections further using a custom **detections_manager** function that processes zone-based transitions to determine vehicle turns.

6. Annotates the frame (e.g., drawing bounding boxes and turn labels) using the **annotate_frame** function.

In [13]:

def process_frame(frame: np.ndarray, config: Dict[str, Any]) -> np.ndarray:
    result = config["model"](frame, verbose=False, conf=config["conf_threshold"], iou=config["iou_threshold"])[0]
    detections = sv.Detections.from_ultralytics(result)
    detections.class_id = np.zeros(len(detections))
    detections = config["tracker"].update_with_detections(detections)
    detections_in_zones, detections_out_zones = [], []
    for zone_in, zone_out in zip(config["zones_in"].values(), config["zones_out"].values()):
        in_zone = detections[zone_in.trigger(detections)]
        out_zone = detections[zone_out.trigger(detections)]
        detections_in_zones.append(in_zone)
        detections_out_zones.append(out_zone)

    filtered = config["detections_manager"](detections, detections_in_zones, detections_out_zones,config)
    return annotate_frame(frame, filtered, config)

# Append Turn Analysis Summary Chart to Video Output

The function add_final_summary_to_video enhances a processed video by appending a visual summary of vehicle turn statistics at the end. Here's what it does:

1. Analyzes turn data using the provided vehicle turn state.

2. Loads a bar chart image (turn_analysis.png) that visually represents turn statistics.

3. Reads and copies all frames from the original processed video.

4. Appends the chart image as static frames for 5 seconds at the end of the video.

5. Saves the new video with the summary chart to the specified output path.

6. Returns a structured JSON summary of the vehicle turn data.


In [14]:

def add_final_summary_to_video(video_path, vehicle_turns, output_path="final_output.mp4"):
    """Add a final summary frame to the end of the video"""

    # First analyze the turns
    vehicle_turn_json = analyze_turns(vehicle_turns)

    # Load the bar chart image
    chart_img = cv2.imread("turn_analysis.png")
    if chart_img is None:
        raise FileNotFoundError("turn_analysis.png not found.")

    # Read the original video
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Resize chart to match video resolution
    chart_img = cv2.resize(chart_img, (width, height))

    # Create the output video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') #mp4v h264
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # Copy all frames from the original video
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        out.write(frame)

    # Append chart image as 5 seconds of frames
    for _ in range(int(fps * 5)):
        out.write(chart_img)

    # Release resources
    cap.release()
    out.release()
    print(f"Final video with chart saved as '{output_path}'")

    return vehicle_turn_json

# Init Video Processing (Full Video-Based Vehicle Turn Detection and Summary Pipeline)

This function **run_full_vehicle_turn_pipeline** performs the complete pipeline for analyzing vehicle movements in a video. It:

1. Processes the input video using a configured video processor to detect and trace vehicle movements.

2. Analyzes vehicle turns (left, right, U-turn, straight) and records them.

3. Generates a summary chart of the turn statistics and appends it to the output video.

4. Returns a JSON summary of vehicle turn analytics for further use (e.g., visualization or question answering).


In [15]:

def run_full_vehicle_turn_pipeline(
    source_video_path: str,
    final_output_path: str = "final_output.mp4"
):
    """
    Runs the full pipeline: processes video, tracks turns, and appends summary.
    """
    # Step 1: Setup and process the video
    config = setup_video_processor(
        source_video_path=source_video_path,
        target_video_path="output_traced.mp4"
    )
    vehicle_turns_state = process_video(config)

    # Step 2: Append summary chart to the traced video
    vehicle_turn_json = add_final_summary_to_video(
        video_path="output_traced.mp4",
        vehicle_turns=vehicle_turns_state,
        output_path=final_output_path
    )
    return vehicle_turn_json


# Vehicle Turn Detection Summary & AI-Powered Question Answering.
**convert_turn_stats_to_text(analysis_result)**:
Converts the vehicle turn detection results (a JSON dictionary) into a readable text summary, including:

1. Total vehicle count

2. Turn type counts (right, left, U-turn, straight)

3. Per-vehicle turn information.

**Use:**
This summary is later passed to a language model for answering questions.

**create_pipeline(text_data)**:
Creates a custom question-answering function qa_pipeline(question) that:

1. Takes a natural language question

2. Feeds it to Qwen along with the vehicle turn summary

3. Returns only the assistant's reply from the model output

**Purpose:**
This abstracts the model usage so the user can ask follow-up questions based on video analytics.

In [16]:
def convert_turn_stats_to_text(analysis_result):
    turn_counts = analysis_result.get("turn_counts", {})
    turn_details = analysis_result.get("turn_details", [])

    summary_text = (
        f"Total Vehicles Tracked: {analysis_result.get('total_vehicles', 0)}\n"
        f"Vehicles Right Turns: {turn_counts.get('Vehicles making right turns', 0)}\n"
        f"Vehicles Left Turns: {turn_counts.get('Vehicles making left turns', 0)}\n"
        f"Vehicles U turns: {turn_counts.get('Vehicles making U-turns', 0)}\n"
        f"Vehicles with no detected turns or Straight or no turns: {turn_counts.get('Vehicles with no detected turns (Straight)', 0)}\n"
    )

    detail_lines = [
        f"Tracker / Vehicle ID {item['tracker_id']}: {item['turn'].replace('_', ' ').capitalize()}"
        for item in turn_details
    ]

    details_text = "\n".join(detail_lines)

    return f"{summary_text}\nIndividual Vehicle Turns:\n{details_text}"



# Load Qwen model and tokenizer (only once globally)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct")
generation_pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)



def create_pipeline(text_data):
    """
    Create a simple function to handle QA using Qwen with the full text_data
    """
    def qa_pipeline(question):
        text = f"""
        You are an expert in analyzing traffic video data, specifically vehicle turn behavior.

        Context:
        {text_data}

        Based on the context above, answer the following question clearly and concisely.

        Question:
        {question}
        """

        messages = [
            {"role": "system", "content": "You are an expert on vehicle turn analysis. Respond clearly using the data provided."},
            {"role": "user", "content": text},
        ]
        response = generation_pipe(messages, max_new_tokens=1000)[0]
        print(response)
        assistant_response = ""
        for msg in response['generated_text']:
            if msg.get("role") == "assistant":
                assistant_response = msg.get("content", "")
                break
        return assistant_response

    return qa_pipeline


Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Device set to use cuda:0


# Main execution and Gradio UI

---
This Code Block allows users to:

1. Upload a video for vehicle turn analysis.

2. View a processed version of the video.

3. Ask natural language questions (e.g., "How many U-turns?") and receive answers from an LLM based on the analysis.

Explanation:

---
**gradio** is used to create an interactive web UI for uploading a video, processing it, and asking questions.

**tempfile** is used to handle temporary storage of the uploaded video.

global variables store:

*  **global_turn_json**: the analysis results (as a dictionary).
*  **global_pipeline:** the question-answering pipeline based on the analysis text.

Analysis Video Block: (**Took 500 seconds to process**)

1. Reads the uploaded video and saves it temporarily.

2. Runs the vehicle turn detection pipeline (**run_full_vehicle_turn_pipeline**).

3. Converts the resulting data to a readable summary using **convert_turn_stats_to_text**.

4. Sets up the question-answering pipeline with **create_pipeline**.

5. Returns the path to the processed video and a status message.






In [17]:
import gradio as gr
import tempfile
import subprocess

# Persistent state for document store and pipeline

global_pipeline = None
global_turn_json = None

def analyze_video(video_file_path):
    global global_pipeline, global_turn_json

    if not video_file_path:
        return None, "Please upload a video file."

    with open(video_file_path, "rb") as source_file:
        with tempfile.NamedTemporaryFile(delete=False, suffix=".mp4") as tmp_file:
            tmp_file.write(source_file.read())
            tmp_video_path = tmp_file.name

    raw_output_path = "raw_output.mp4"
    browser_safe_path = "final_output_with_summary.mp4"

    # Run the full vehicle turn detection pipeline
    global_turn_json = run_full_vehicle_turn_pipeline(
        source_video_path=tmp_video_path,
        final_output_path=raw_output_path
    )

    # Re-encode video
    encode_to_browser_safe_mp4(raw_output_path, browser_safe_path)

    # Create document store and pipeline for QA
    text_data = convert_turn_stats_to_text(global_turn_json)
    global_pipeline = create_pipeline(text_data)


    return browser_safe_path, "Video analyzed successfully! You can now ask questions."

def answer_question(user_question):
    if not global_pipeline or not global_turn_json:
        return "Please analyze a video first."

    return global_pipeline(user_question)

# Gradio UI
video_input = gr.Video(label="Upload a video")
analyze_btn = gr.Button("Submit & Analyze")
video_output = gr.Video(label="Processed Video")
status_output = gr.Textbox(label="Status")

question_input = gr.Textbox(label="Ask a question (e.g., How many U-turns were made?)")
answer_output = gr.Textbox(label="Answer")

with gr.Blocks(title="Vehicle Turn Detection and QA") as demo:
    gr.Markdown("## Vehicle Turn Detection and Q&A")
    with gr.Row():
        with gr.Column():
            video_input.render()
            analyze_btn.render()
        with gr.Column():
            video_output.render()
            status_output.render()

    analyze_btn.click(fn=analyze_video, inputs=video_input, outputs=[video_output, status_output])

    gr.Markdown("### Ask a question after analysis:")
    question_input.render()
    answer_output.render()

    question_input.change(fn=answer_question, inputs=question_input, outputs=answer_output)



def encode_to_browser_safe_mp4(input_path: str, output_path: str):
    cmd = [
        "ffmpeg", "-y",
        "-i", input_path,
        "-vcodec", "libx264",
        "-acodec", "aac",
        "-movflags", "+faststart",  # enables progressive download
        output_path
    ]
    try:
        subprocess.run(cmd, check=True)
    except subprocess.CalledProcessError:
        print("Error: ffmpeg failed to convert video to browser-safe format.")

if __name__ == "__main__":
    demo.launch(debug=True)



--- Turn Analysis Results ---
Total unique vehicles tracked: 69
Vehicles making right turns: 13 
Vehicles making left turns: 15
Vehicles making U-turns: 4
Vehicles with no detected turns: 37
Final video with chart saved as 'raw_output.mp4'
{'generated_text': [{'role': 'system', 'content': 'You are an expert on vehicle turn analysis.'}, {'role': 'user', 'content': '\n        Context:Total Vehicles Tracked: 69\nVehicles Right Turns: 13\nVehicles Left Turns: 15\nVehicles U turns: 4\nVehicles with no detected turns or Straight or no turns: 37\n\nIndividual Vehicle Turns:\nTracker / Vehicle ID 25: Straight\nTracker / Vehicle ID 20: Straight\nTracker / Vehicle ID 48: Straight\nTracker / Vehicle ID 95: Left turn\nTracker / Vehicle ID 69: Straight\nTracker / Vehicle ID 75: Right turn\nTracker / Vehicle ID 59: Straight\nTracker / Vehicle ID 76: Straight\nTracker / Vehicle ID 144: Left turn\nTracker / Vehicle ID 10: Straight\nTracker / Vehicle ID 9: Straight\nTracker / Vehicle ID 13: Right turn

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


{'generated_text': [{'role': 'system', 'content': 'You are an expert on vehicle turn analysis.'}, {'role': 'user', 'content': '\n        Context:Total Vehicles Tracked: 69\nVehicles Right Turns: 13\nVehicles Left Turns: 15\nVehicles U turns: 4\nVehicles with no detected turns or Straight or no turns: 37\n\nIndividual Vehicle Turns:\nTracker / Vehicle ID 25: Straight\nTracker / Vehicle ID 20: Straight\nTracker / Vehicle ID 48: Straight\nTracker / Vehicle ID 95: Left turn\nTracker / Vehicle ID 69: Straight\nTracker / Vehicle ID 75: Right turn\nTracker / Vehicle ID 59: Straight\nTracker / Vehicle ID 76: Straight\nTracker / Vehicle ID 144: Left turn\nTracker / Vehicle ID 10: Straight\nTracker / Vehicle ID 9: Straight\nTracker / Vehicle ID 13: Right turn\nTracker / Vehicle ID 21: Straight\nTracker / Vehicle ID 206: Left turn\nTracker / Vehicle ID 135: Straight\nTracker / Vehicle ID 156: Straight\nTracker / Vehicle ID 23: Straight\nTracker / Vehicle ID 177: Straight\nTracker / Vehicle ID 186